In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import os
import chardet
import seaborn as sns  
from numpy import isnan
from pandas import read_csv
from sklearn.impute import KNNImputer
from mlxtend.preprocessing import minmax_scaling # for min_max scaling

In [ ]:
# read in all our data
covid_data = pd.read_csv("/content/covid_analytics_clinical_data (4).csv")

# set seed for reproducibility
np.random.seed(0) 

In [ ]:
covid_data.head()

,ID,SUB_ID,Country,Province/State,Study Pop Size (N),Proportion of values entered in row (%),Paper Title,Journal,Author,Source,Date Added,Language,Peer-Reviewed? (As of Date Added),Study Type: Observational or Randomized Clinical Trial (RCT)?,Study start date (mm/dd/yyyy),Study end date (mm/dd/yyyy),Study length (days),Overall study population or subgroup?,Subgroup,Positive/negative cases,End-point vs Active,Survivors,Discharged,Contains Any Treatment?,ICU vs. non-ICU,Severity,Ventilation,Median Age,Mean Age,Age SD,Age LQ,Age UQ,% Healthcare Worker,% Male,% White or European,% African American,% Asian,% Hispanic or Latino,% Multiple ethnicities or other,Exposure history (%),...,Uses Kaletra (lopinavir–ritonavir),Uses Favipiravir,Uses Tamiflu (oseltamivir),Uses Remdesivir,Uses umif+lop-rit,Uses Arbidol (umifenovir),Uses hydroxychloroquine and/or chloroquine,"Corticosteroid (including Glucocorticoid, Methylprednisolone)",Intravenous immunoglobin,Nasal cannula,High-flow nasal cannula oxygen therapy,Oxygen therapy,Noninvasive mechanical ventilation,Invasive mechanical ventilation,ECMO,Renal replacement therapy,Interferon Alpha-1b,Thymalfasin and/or Thymosin,Sepsis,Respiratory failure or ARDS,Respiratory failure,ARDS,Hypoxemia,Heart failure,Septic shock / shock,Liver dysfunction,Coagulopathy,Acute cardiac injury,Acute kidney injury (AKI),Secondary infection/ Bacterial infection,Hypoproteinaemia,Acidosis,Hospital admission (%),ICU admission,Discharged (%),"ICU length of stay, days","Hospital length of stay, days",Days to Viral Clearance (Median),Mortality,Projected Mortality (accounting for patients not currently discharged)
0,1.0,0.0,China,Wuhan,191.0,60.11%,Clinical course and risk factors for mortality...,Lancet,Zhou et al.,https://www.thelancet.com/journals/lancet/arti...,4/2/20,English,Yes,Observational,12/29/19,1/31/20,33.0,Top-level,General,Positive only,End-point only,Both,Both,Yes,Both,All,Both,56.0,NaN,NaN,46.0,67.0,NaN,62.30%,0%,0%,100%,0%,0%,38%,...,21.47%,0.00%,0.00%,NaN,0.00%,0.00%,0.00%,29.84%,24.08%,NaN,21.47%,NaN,13.61%,16.75%,1.57%,5.24%,0.00%,NaN,58.64%,NaN,53.93%,30.89%,NaN,23.04%,19.90%,NaN,19.37%,17.28%,14.66%,14.66%,11.52%,8.90%,1.0,0.26,0.72,8,11,20.0,0.28,0.28
1,1.0,1.0,China,Wuhan,54.0,60.11%,Clinical course and risk factors for mortality...,Lancet,Zhou et al.,https://www.thelancet.com/journals/lancet/arti...,4/2/20,English,Yes,Observational,12/29/19,1/31/20,33.0,Subgroup,Non-survivors,Positive only,End-point only,Non-survivors only,Discharged only,Yes,Both,Severe/Critical Only,Both,69.0,NaN,NaN,63.0,76.0,NaN,70.37%,0%,0%,100%,0%,0%,25.93%,...,22.22%,0.00%,0.00%,NaN,0.00%,0.00%,0.00%,48.15%,66.67%,NaN,61.11%,NaN,44.44%,57.41%,5.56%,18.52%,0.00%,NaN,100%,NaN,98.15%,92.59%,NaN,51.85%,70.37%,NaN,50.00%,59.26%,50.00%,50.00%,37.04%,29.63%,1.0,0.72,0,8,7.5,7.5,1,1
2,1.0,2.0,China,Wuhan,137.0,60.11%,Clinical course and risk factors for mortality...,Lancet,Zhou et al.,https://www.thelancet.com/journals/lancet/arti...,4/2/20,English,Yes,Observational,12/29/19,1/31/20,33.0,Subgroup,Survivors,Positive only,End-point only,Survivors only,Discharged only,Yes,Both,All,Both,52.0,NaN,NaN,45.0,58.0,NaN,59.12%,0%,0%,100%,0%,0%,43.07%,...,21.27%,0.00%,0.00%,NaN,0.00%,0.00%,0.00%,22.63%,7.30%,NaN,5.84%,NaN,1.46%,0.73%,0.00%,0.00%,0.00%,NaN,42.34%,NaN,36.50%,6.57%,NaN,11.68%,0.00%,NaN,7.30%,0.73%,0.73%,0.73%,1.46%,0.73%,1.0,0.08,1,7,12,20.0,0,0
3,2.0,0.0,China,Wuhan,199.0,44.94%,A Trial of Lopinavir–Ritonavir in Adults Hospi...,NEJM,Cao et. al.,https://www.nejm.org/doi/full/10.1056/NEJMoa20...,4/2/20,English,Yes,RCT,1/18/20,2/3/20,16.0,Top-level,General,Positive only,End-point and active,Both,Both,Yes,Both,All,Both,58.0,NaN,NaN,49.0,68.0,NaN,60.30%,0%,0%,100%,0%,0%,NaN,...,47%,0%,0%,NaN,0%,0%,0%,33.70%,NaN,NaN,NaN,NaN,14.60%,16.10%,2%,4.50%,NaN,NaN,0.49%,20.10%,NaN,NaN,NaN,0.51%,2.05%,NaN,NaN,NaN,4.68%,3.65%,NaN,NaN,1.0,NaN,0.72,10,15,28.0,0.16,0.18
4,2.0,1.0,China,Wuhan,99.0,44.94%,A Trial of Lopinavir–Ritonavir in Adults Hospi

In [ ]:
#dropping the columns that are not required in the calculation
covid_data = covid_data.drop(['Paper Title','Journal','Author','Source','Date Added','Language','Peer-Reviewed? (As of Date Added)','Study start date (mm/dd/yyyy)','Study end date (mm/dd/yyyy)','% White or European','% African American','% Asian','% Hispanic or Latino','% Multiple ethnicities or other','Mortality'], axis=1)
covid_data.head()

,ID,SUB_ID,Country,Province/State,Study Pop Size (N),Proportion of values entered in row (%),Study Type: Observational or Randomized Clinical Trial (RCT)?,Study length (days),Overall study population or subgroup?,Subgroup,Positive/negative cases,End-point vs Active,Survivors,Discharged,Contains Any Treatment?,ICU vs. non-ICU,Severity,Ventilation,Median Age,Mean Age,Age SD,Age LQ,Age UQ,% Healthcare Worker,% Male,Exposure history (%),Exposure - Wuhan Travel (%),Exposure - Personal Contact (%),Incubation period (days) - Mean,Incubation period (days) - Median,Incubation period (days) - LQ,Incubation period (days) - Higher Quartile,Time From Symptoms Appearance to Hospitalization (days) - Mean,Time From Symptoms Appearance to Hospitalization (days) - Median,Time From Symptoms Appearance to Hospitalization (days) - LQ,Time From Symptoms Appearance to Hospitalization (days) - UQ,Smoking history,Current drinker,BMI,Obesity,...,Antiviral (Any),Uses Kaletra (lopinavir–ritonavir),Uses Favipiravir,Uses Tamiflu (oseltamivir),Uses Remdesivir,Uses umif+lop-rit,Uses Arbidol (umifenovir),Uses hydroxychloroquine and/or chloroquine,"Corticosteroid (including Glucocorticoid, Methylprednisolone)",Intravenous immunoglobin,Nasal cannula,High-flow nasal cannula oxygen therapy,Oxygen therapy,Noninvasive mechanical ventilation,Invasive mechanical ventilation,ECMO,Renal replacement therapy,Interferon Alpha-1b,Thymalfasin and/or Thymosin,Sepsis,Respiratory failure or ARDS,Respiratory failure,ARDS,Hypoxemia,Heart failure,Septic shock / shock,Liver dysfunction,Coagulopathy,Acute cardiac injury,Acute kidney injury (AKI),Secondary infection/ Bacterial infection,Hypoproteinaemia,Acidosis,Hospital admission (%),ICU admission,Discharged (%),"ICU length of stay, days","Hospital length of stay, days",Days to Viral Clearance (Median),Projected Mortality (accounting for patients not currently discharged)
0,1.0,0.0,China,Wuhan,191.0,60.11%,Observational,33.0,Top-level,General,Positive only,End-point only,Both,Both,Yes,Both,All,Both,56.0,NaN,NaN,46.0,67.0,NaN,62.30%,38%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,8.0,14.0,5.76%,NaN,NaN,NaN,...,21.47%,21.47%,0.00%,0.00%,NaN,0.00%,0.00%,0.00%,29.84%,24.08%,NaN,21.47%,NaN,13.61%,16.75%,1.57%,5.24%,0.00%,NaN,58.64%,NaN,53.93%,30.89%,NaN,23.04%,19.90%,NaN,19.37%,17.28%,14.66%,14.66%,11.52%,8.90%,1.0,0.26,0.72,8,11,20.0,0.28
1,1.0,1.0,China,Wuhan,54.0,60.11%,Observational,33.0,Subgroup,Non-survivors,Positive only,End-point only,Non-survivors only,Discharged only,Yes,Both,Severe/Critical Only,Both,69.0,NaN,NaN,63.0,76.0,NaN,70.37%,25.93%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,8.0,15.0,9.26%,NaN,NaN,NaN,...,22.22%,22.22%,0.00%,0.00%,NaN,0.00%,0.00%,0.00%,48.15%,66.67%,NaN,61.11%,NaN,44.44%,57.41%,5.56%,18.52%,0.00%,NaN,100%,NaN,98.15%,92.59%,NaN,51.85%,70.37%,NaN,50.00%,59.26%,50.00%,50.00%,37.04%,29.63%,1.0,0.72,0,8,7.5,7.5,1
2,1.0,2.0,China,Wuhan,137.0,60.11%,Observational,33.0,Subgroup,Survivors,Positive only,End-point only,Survivors only,Discharged only,Yes,Both,All,Both,52.0,NaN,NaN,45.0,58.0,NaN,59.12%,43.07%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,8.0,13.0,4.38%,NaN,NaN,NaN,...,21.17%,21.27%,0.00%,0.00%,NaN,0.00%,0.00%,0.00%,22.63%,7.30%,NaN,5.84%,NaN,1.46%,0.73%,0.00%,0.00%,0.00%,NaN,42.34%,NaN,36.50%,6.57%,NaN,11.68%,0.00%,NaN,7.30%,0.73%,0.73%,0.73%,1.46%,0.73%,1.0,0.08,1,7,12,20.0,0
3,2.0,0.0,China,Wuhan,199.0,44.94%,RCT,16.0,Top-level,General,Positive only,End-point and active,Both,Both,Yes,Both,All,Both,58.0,NaN,NaN,49.0,68.0,NaN,60.30%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,16.0,NaN,NaN,NaN,NaN,...,47%,47%,0%,0%,NaN,0%,0%,0%,33.70%,NaN,NaN,NaN,NaN,14.60%,16.10%,2%,4.50%,NaN,NaN,0.49%,20.10%,NaN,NaN,NaN,0.51%,2.05%,NaN,NaN,NaN,4.68%,3.65%,NaN,NaN,1.0,NaN,0.72,10,15,28.0,0.18
4,2.0,1.0,China,Wuhan,99.0,44.94%,RCT,16.0,Subgroup,With treatment,Positive only,End-point and active,Both,Both,Yes,Both,All,Both,58.0,NaN,NaN,50.0,68.0,NaN,61.60%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,17.0,NaN,NaN,NaN,NaN,...,94.90%,94.90%,0%,0%,NaN,0%,0%,

In [ ]:
# percent of data that is missing to figure out the scale of the problem
total_cells = np.product(covid_data.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

61.570305302083725


In [ ]:
all_columns = list(covid_data)
print(all_columns)
covid_data['Proportion of values entered in row (%)'] = covid_data['Proportion of values entered in row (%)'].str.rstrip('%').astype('float') / 100.0
covid_data['% Healthcare Worker'] = covid_data['% Healthcare Worker'].str.rstrip('%').astype('float') / 100.0
covid_data['% Male'] = covid_data['% Male'].str.rstrip('%').astype('float') / 100.0
covid_data['Exposure history (%)'] = covid_data['Exposure history (%)'].str.rstrip('%').astype('float') / 100.0
covid_data['Exposure - Wuhan Travel (%)'] = covid_data['Exposure - Wuhan Travel (%)'].str.rstrip('%').astype('float') / 100.0
covid_data['Exposure - Personal Contact (%)'] = covid_data['Exposure - Personal Contact (%)'].str.rstrip('%').astype('float') / 100.0

['ID', 'SUB_ID', 'Country', 'Province/State', 'Study Pop Size (N)', 'Proportion of values entered in row (%)', 'Study Type: Observational or Randomized Clinical Trial (RCT)?', 'Study length (days)', 'Overall study population or subgroup?', 'Subgroup', 'Positive/negative cases', 'End-point vs Active', 'Survivors', 'Discharged', 'Contains Any Treatment?', 'ICU vs. non-ICU', 'Severity', 'Ventilation', 'Median Age', 'Mean Age', 'Age SD', 'Age LQ', 'Age UQ', '% Healthcare Worker', '% Male', 'Exposure history (%)', 'Exposure - Wuhan Travel (%)', 'Exposure - Personal Contact (%)', 'Incubation period (days) - Mean', 'Incubation period (days) - Median', 'Incubation period (days) - LQ', 'Incubation period (days) -  Higher Quartile', 'Time From Symptoms Appearance to Hospitalization (days) - Mean', 'Time From Symptoms Appearance to Hospitalization (days) - Median', 'Time From Symptoms Appearance to Hospitalization (days) - LQ', 'Time From Symptoms Appearance to Hospitalization (days) - UQ', 'Smok

In [ ]:
# getting the number of missing data points per column
missing_values_count = covid_data.isnull().sum()

# look at the number of missing points in the first ten columns
missing_values_count[0:10]

ID                                                                1
SUB_ID                                                            1
Country                                                           1
Province/State                                                   20
Study Pop Size (N)                                                1
Proportion of values entered in row (%)                           1
Study Type: Observational or Randomized Clinical Trial (RCT)?    25
Study length (days)                                               4
Overall study population or subgroup?                             1
Subgroup                                                          7
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
for j in covid_data.columns.values:
    le = LabelEncoder()
### fit with the desired col, col in position 0 for this ###example
    fit_by = pd.Series([i for i in covid_data[j].unique() if type(i) == str])
    le.fit(fit_by)
    ### Set transformed col leaving np.NaN as they are
    covid_data[j] = covid_data[j].apply(lambda x: le.transform([x])[0] if type(x) == str else x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [ ]:
covid_data.head(10)

,ID,SUB_ID,Country,Province/State,Study Pop Size (N),Proportion of values entered in row (%),Study Type: Observational or Randomized Clinical Trial (RCT)?,Study length (days),Overall study population or subgroup?,Subgroup,Positive/negative cases,End-point vs Active,Survivors,Discharged,Contains Any Treatment?,ICU vs. non-ICU,Severity,Ventilation,Median Age,Mean Age,Age SD,Age LQ,Age UQ,% Healthcare Worker,% Male,Exposure history (%),Exposure - Wuhan Travel (%),Exposure - Personal Contact (%),Incubation period (days) - Mean,Incubation period (days) - Median,Incubation period (days) - LQ,Incubation period (days) - Higher Quartile,Time From Symptoms Appearance to Hospitalization (days) - Mean,Time From Symptoms Appearance to Hospitalization (days) - Median,Time From Symptoms Appearance to Hospitalization (days) - LQ,Time From Symptoms Appearance to Hospitalization (days) - UQ,Smoking history,Current drinker,BMI,Obesity,...,Antiviral (Any),Uses Kaletra (lopinavir–ritonavir),Uses Favipiravir,Uses Tamiflu (oseltamivir),Uses Remdesivir,Uses umif+lop-rit,Uses Arbidol (umifenovir),Uses hydroxychloroquine and/or chloroquine,"Corticosteroid (including Glucocorticoid, Methylprednisolone)",Intravenous immunoglobin,Nasal cannula,High-flow nasal cannula oxygen therapy,Oxygen therapy,Noninvasive mechanical ventilation,Invasive mechanical ventilation,ECMO,Renal replacement therapy,Interferon Alpha-1b,Thymalfasin and/or Thymosin,Sepsis,Respiratory failure or ARDS,Respiratory failure,ARDS,Hypoxemia,Heart failure,Septic shock / shock,Liver dysfunction,Coagulopathy,Acute cardiac injury,Acute kidney injury (AKI),Secondary infection/ Bacterial infection,Hypoproteinaemia,Acidosis,Hospital admission (%),ICU admission,Discharged (%),"ICU length of stay, days","Hospital length of stay, days",Days to Viral Clearance (Median),Projected Mortality (accounting for patients not currently discharged)
0,1.0,0.0,2.0,48.0,191.0,0.6011,0.0,33.0,1.0,88.0,4.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,56.0,NaN,NaN,46.0,67.0,NaN,0.6230,0.3800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,8.0,14.0,90.0,NaN,NaN,NaN,...,7.0,17.0,2.0,1.0,NaN,1.0,1.0,1.0,40.0,32.0,NaN,16.0,NaN,23.0,37.0,5.0,40.0,1.0,NaN,18.0,NaN,21.0,43.0,NaN,28.0,25.0,NaN,9.0,16.0,21.0,13.0,3.0,6.0,1.0,0.26,54.0,16.0,5.0,20.0,27.0
1,1.0,1.0,2.0,48.0,54.0,0.6011,0.0,33.0,0.0,140.0,4.0,4.0,1.0,3.0,1.0,0.0,7.0,0.0,69.0,NaN,NaN,63.0,76.0,NaN,0.7037,0.2593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,8.0,15.0,117.0,NaN,NaN,NaN,...,8.0,18.0,2.0,1.0,NaN,1.0,1.0,1.0,70.0,70.0,NaN,44.0,NaN,73.0,77.0,20.0,19.0,1.0,NaN,11.0,NaN,28.0,76.0,NaN,38.0,53.0,NaN,16.0,32.0,71.0,34.0,5.0,5.0,1.0,0.72,0.0,16.0,81.0,7.5,59.0
2,1.0,2.0,2.0,48.0,137.0,0.6011,0.0,33.0,0.0,209.0,4.0,4.0,2.0,3.0,1.0,0.0,0.0,0.0,52.0,NaN,NaN,45.0,58.0,NaN,0.5912,0.4307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,8.0,13.0,77.0,NaN,NaN,NaN,...,6.0,16.0,2.0,1.0,NaN,1.0,1.0,1.0,29.0,73.0,NaN,41.0,NaN,10.0,8.0,1.0,1.0,1.0,NaN,17.0,NaN,16.0,65.0,NaN,16.0,2.0,NaN,18.0,5.0,9.0,6.0,2.0,2.0,1.0,0.08,80.0,15.0,11.0,20.0,0.0
3,2.0,0.0,2.0,48.0,199.0,0.4494,1.0,16.0,1.0,88.0,4.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,58.0,NaN,NaN,49.0,68.0,NaN,0.6030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,16.0,NaN,NaN,NaN,NaN,...,24.0,29.0,1.0,0.0,NaN,0.0,0.0,0.0,47.0,NaN,NaN,NaN,NaN,26.0,35.0,13.0,34.0,NaN,NaN,6.0,18.0,NaN,NaN,NaN,7.0,27.0,NaN,NaN,NaN,58.0,24.0,NaN,NaN,1.0,NaN,54.0,2.0,25.0,28.0,18.0
4,2.0,1.0,2.0,48.0,99.0,0.4494,1.0,16.0,0.0,235.0,4.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,58.0,NaN,NaN,50.0,68.0,NaN,0.6160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,17.0,NaN,NaN,NaN,NaN,...,77.0,58.0,1.0,0.0,NaN,0.0,0.0,0.0,45.0,NaN,NaN,NaN,NaN,13.0,30.0,13.0,29.0,NaN,NaN,1.0,10.0,NaN,NaN,NaN,1.0,28.0,NaN,NaN,NaN,45.0,7.0,NaN,NaN,1.0,NaN,56.0,14.0,21.0,28.0,17.0
5,2.0,2.0,2.0,48.0,100.0,0.4494,1.0,16.0,0.0,71.0,4.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,58.0,NaN,NaN,48.0,68.0,NaN,0.5900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,10.0,16.0,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,49.0,NaN,NaN,NaN,NaN,36.0,38.0,13.0,46.0,NaN,NaN,7.0,21.0,NaN,NaN,NaN,8.0

In [ ]:
Y = covid_data['Projected Mortality (accounting for patients not currently discharged)']
X = covid_data.drop(['Projected Mortality (accounting for patients not currently discharged)'], axis=1)

    ID  SUB_ID  ...  Hospital length of stay, days  Days to Viral Clearance (Median)
0  1.0     0.0  ...                            5.0                              20.0
1  1.0     1.0  ...                           81.0                               7.5
2  1.0     2.0  ...                           11.0                              20.0
3  2.0     0.0  ...                           25.0                              28.0
4  2.0     1.0  ...                           21.0                              28.0

[5 rows x 146 columns]
0    27.0
1    59.0
2     0.0
3    18.0
4    17.0
Name: Projected Mortality (accounting for patients not currently discharged), dtype: float64


In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
covid_data = imputer.fit_transform(covid_data)

In [ ]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.layers import *
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=5)

In [ ]:
input = Input((X_train.shape[0],X_train.shape[1]))
input = K.cast(input, dtype='int32') 
print(input)

Tensor("Cast:0", shape=(None, 431, 146), dtype=int32)


In [ ]:
# Building the model
model = Sequential()

#Input Layer
model.add(Dense(128, kernel_initializer='normal', input_dim = 146, activation='relu'))

#Hidden Layers
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dense(256, kernel_initializer='normal', activation='relu'))

#Output Layer
model.add(Dense(1, kernel_initializer='normal', activation='linear'))

#Compiling the network
model.compile(loss="mean_absolute_error", optimizer ='adam', metrics=["mean_absolute_error"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               18816     
_________________________________________________________________
dense_10 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_12 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 257       
Total params: 183,681
Trainable params: 183,681
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
14/14 [==============================] - 0s 5ms/step - loss: nan - mean_absolute_error: nan
Epoch 2/10
14/14 [==============================] - 0s 3ms/step - loss: nan - mean_absolute_error: nan
Epoch 3/10
14/14 [==============================] - 0s 4ms/step - loss: nan - mean_absolute_error: nan
Epoch 4/10
14/14 [==============================] - 0s 4ms/step - loss: nan - mean_absolute_error: nan
Epoch 5/10
14/14 [==============================] - 0s 3ms/step - loss: nan - mean_absolute_error: nan
Epoch 6/10
14/14 [==============================] - 0s 3ms/step - loss: nan - mean_absolute_error: nan
Epoch 7/10
14/14 [==============================] - 0s 3ms/step - loss: nan - mean_absolute_error: nan
Epoch 8/10
14/14 [==============================] - 0s 3ms/step - loss: nan - mean_absolute_error: nan
Epoch 9/10
14/14 [==============================] - 0s 3ms/step - loss: nan - mean_absolute_error: nan
Epoch 10/10
14/14 [==============================] - 0s 3ms/step - loss: 